##  ParticleNet for trackster-level graph

Input:
- graph of the full event
- edges built between tracksters in a feasible neighborhood 
    - list of edges
- label 1 or 0 for each edge

Explored options:
- GNN with EdgeConv operators to extract information from the neighborhood
    - then predict binary output per edge
- DGCNN/ParticleNet with DynamicEdgeConv
    - let the network make its own edges in the latent space
    - use the original edges as a query after the last conv layer

Dimensionality:
- Using the batch trick to encode multiple samples at once
    - need to reindex edges
    - automatically handled by torch-geometric dataloader

In [32]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import random_split
from torch.optim.lr_scheduler import CosineAnnealingLR

import numpy as np
import matplotlib.pyplot as plt

from torch_geometric.nn import EdgeConv, DynamicEdgeConv
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
import torch_geometric.utils as geo_utils

import sklearn.metrics as metrics

from reco.training import train_edge_pred, test_edge_pred
from reco.dataset import D_TracksterGraph
from reco.loss import FocalLoss

ds_name = "CloseByTwoPion"

data_root = "data"
raw_dir = f"/Users/ecuba/data/{ds_name}"

# data_root = "/mnt/ceph/users/ecuba/processed"
# raw_dir = f"/mnt/ceph/users/ecuba/{ds_name}"

In [33]:
transform = T.Compose([T.NormalizeFeatures()])

ds = D_TracksterGraph(
    ds_name,
    data_root,
    raw_dir,
    N_FILES=1,
    MAX_DISTANCE=1000,
    include_graph_features=False,
    transform=transform,
)
ds

TrackstersGraph(graphs=100, nodes=2960, edges=24787, max_distance=1000, graph_features=False)

In [34]:
ds_size = len(ds)
test_set_size = ds_size // 10
train_set_size = ds_size - test_set_size
train_set, test_set = random_split(ds, [train_set_size, test_set_size])
print(f"Train graphs: {len(train_set)}, Test graphs: {len(test_set)}")

# this is very nice - handles the dimensions automatically
train_dl = DataLoader(train_set, batch_size=64, shuffle=True)
test_dl = DataLoader(test_set, batch_size=64, shuffle=True)

Train graphs: 90, Test graphs: 10


In [35]:
print(f"dataset balance: {float(sum(ds.data.y) / len(ds.data.y)):.3f}") 

dataset balance: 0.733


In [36]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [57]:
# ParticleNet

class EdgeConvBlock(nn.Module):

    def __init__(self, input_dim, hidden_dim, aggr="add", k=8): # skip_link=False,
        super(EdgeConvBlock, self).__init__()

        convnetwork = nn.Sequential(
            nn.Linear(2 * input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )

        self.graphconv = EdgeConv(nn=convnetwork, aggr=aggr)
        # self.dynamicgraphconv = DynamicEdgeConv(nn=convnetwork, aggr=aggr, k=k)
        # self.skip_link = skip_link
        
    def forward(self, X, edge_index):
        
        #if edge_index is None:
        # H = self.dynamicgraphconv(X)
        #else:
        H = self.graphconv(X, edge_index)

        # if self.skip_link:
        #     return torch.hstack((H, X))

        return H    



class GraphNet(nn.Module):
    def __init__(self, input_dim, output_dim=1, aggr='add', dropout=0.2, skip_link=False):
        """
        skip_link might not make so much difference if the edges are fixed
        """
        
        super(GraphNet, self).__init__()

        hdim1 = 64
        in_dim2 = hdim1 # + input_dim if skip_link else hdim1
        
        hdim2 = 128
        in_dim3 = hdim2 # + in_dim2 if skip_link else hdim2

        hdim3 = 256
        in_dim4 = hdim3 # + in_dim3 if skip_link else hdim3

        # EdgeConv
        self.graphconv1 = EdgeConvBlock(input_dim, hdim1) #, skip_link=skip_link)
        self.graphconv2 = EdgeConvBlock(in_dim2, hdim2) #, skip_link=skip_link)
        self.graphconv3 = EdgeConvBlock(in_dim3, hdim3) #, skip_link=skip_link)

        # Edge features from node embeddings for classification
        self.edgenetwork = nn.Sequential(
            nn.Linear(in_dim4, hdim3),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hdim3, output_dim),
            nn.Sigmoid()
        )
            
    def forward(self, X, edge_index):        
        # (prepared_edges, _) = geo_utils.add_self_loops(edge_index)  
        undirected_index = geo_utils.to_undirected(edge_index)

        H = self.graphconv1(X, undirected_index)
        H = self.graphconv2(H, undirected_index)
        H = self.graphconv3(H, undirected_index)
        
        #src, dst = edge_index | torch.cat([H[src], H[dst]], dim=-1)
        return self.edgenetwork(H).squeeze(-1)

In [22]:
model = GraphNet(input_dim=ds.data.x.shape[1], skip_link=False)
epochs = 50

# loss_func = F.binary_cross_entropy_with_logits
# alpha - percentage of negative edges
loss_func = FocalLoss(alpha=0.75, gamma=2)

model = model.to(device)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=1e-3)

for epoch in range(epochs):

    train_loss, train_true, train_pred = train_edge_pred(
        model,
        device,
        optimizer,
        loss_func,
        train_dl
    )
    
    train_acc = metrics.accuracy_score(train_true, (train_pred > 0.5).astype(int))
    scheduler.step()

    if epoch % 2 == 0:
        test_loss, test_true, test_pred = test_edge_pred(model, device, loss_func, test_dl)
        test_acc = metrics.accuracy_score(test_true, (test_pred > 0.5).astype(int))
        print(
            f"Epoch {epoch}:",
            f"\ttrain loss:{train_loss:.2f}\ttrain acc: {train_acc:.3f}",
            f"\t test loss:{test_loss:.2f} \t test acc: {test_acc:.3f}"
        )

AttributeError: 'GlobalStorage' object has no attribute 'mask'

In [48]:
th_values = [i / 100. for i in range(1, 99)]
precision = []
recall = []
fbeta = []
accuracy = []

for th in th_values:
    test_loss, test_true, test_pred = test_edge_pred(model, device, loss_func, test_dl)

    pred = (test_pred > th).astype(int)

    if sum(pred) == 0:
        precision.append(0)
        recall.append(0)
        fbeta.append(0)
        accuracy.append(0)
    else:
        accuracy.append(metrics.accuracy_score(test_true, pred))
        precision.append(metrics.precision_score(test_true, pred))
        recall.append(metrics.recall_score(test_true, pred))
        fbeta.append(metrics.fbeta_score(test_true, pred, beta=1))

plt.figure()
plt.plot(th_values, precision, label="precision")
plt.plot(th_values, recall, label="recall")
plt.plot(th_values, fbeta, label="fbeta")
plt.plot(th_values, accuracy, label="accuracy")
plt.xlabel("Threshold")
plt.legend()
plt.show()

decision_th = th_values[np.argmax(fbeta)]
print(f"Th: {decision_th} | F-score: {max(fbeta):.3f} | accuracy: {accuracy[np.argmax(fbeta)]:.3f}")

TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
torch.save(model.state_dict(), f"models/DynamicParticleNet_64_128_256_noskip-{epochs}e-CloseByTwoPion_10_10_ngf_{ds.N_FILES}f.pt")

## Scoreboard

### EdgeConv-based

ParticleNet_64_128_256_skip
- Best F-score: 0.902
- Accuracy: 0.937
    
ParticleNet_64_128_256_noskip
- Best F-score: 0.902
- accuracy: 0.937

### DynamicEdgeConv

The knn search in the highly dimensional space makes the training much slower.

DynamicParticleNet_64_128_256_skip
- F-score: 0.880
- accuracy: 0.922
- TH: 0.45




In [54]:
import uproot
import numpy as np
from reco.evaluation import graph_model_evaluation

file_name = f"{raw_dir}/new_ntuples_15101852_158.root"

tracksters = uproot.open({file_name: "ticlNtuplizer/tracksters"})
simtracksters = uproot.open({file_name: "ticlNtuplizer/simtrackstersSC"})
graphs = uproot.open({file_name: "ticlNtuplizer/graph"})
associations = uproot.open({file_name: "ticlNtuplizer/associations"})

In [58]:
model = GraphNet(input_dim=ds.data.x.shape[1]) #$, skip_link=False)

# model.load_state_dict(torch.load(
#     "models/ParticleNet_64_128_256_noskip-100e-CloseByTwoPion_10_10_ngf_500f.pt",
#     map_location=torch.device('cpu')
# ))

In [60]:
import torch.onnx

onnx_filepath = f"onnx/ParticleNet_64_128_256_noskip-CloseByTwoPion_10_10_ngf_{ds.N_FILES}f.onnx"

torch.onnx.export(
    model,                          # model to be exported
    (ds[0].x, ds[0].edge_index),    # example input (add batch dimension)
    onnx_filepath,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['features'],      # the model's input names
    output_names=['output'],    # the model's output names
    dynamic_axes={              # variable length axes
        'features' : {0 : 'batch_size'},    
        'output' : {0 : 'batch_size'}
    }
)

In [ ]:
result = graph_model_evaluation(
    tracksters,
    simtracksters,
    associations,
    graphs,
    model.to("cpu"),
    0.5,
    max_distance=10,
    energy_threshold=10,
    max_events=5,
)

In [ ]:
from reco.dummy import GraphNaiveDummy

result = graph_model_evaluation(
    tracksters,
    simtracksters,
    associations,
    graphs,
    GraphNaiveDummy(),
    0.5,
    max_distance=10,
    energy_threshold=10,
    max_events=5,
)

## Results interpretation

The network is doing a good job in reaching the reconstruction target.
Recall is improved while the precision is kept.

Network performance:
```
mean clue3d_to_sim:	P: 0.96 R: 0.22 F: 0.25
mean target_to_sim:	P: 0.96 R: 0.46 F: 0.51
mean reco_to_sim:	P: 0.96 R: 0.45 F: 0.50
```

Yet, the dataset is too simple as connecting to all the tracksters in the small neighbourhood is feasible without a significant precision loss.

Connecting all tracksters within a small neighborhood:
```
mean clue3d_to_sim:	P: 0.96 R: 0.22 F: 0.25
mean reco_to_sim:	P: 0.95 R: 0.48 F: 0.53
```